### TO-DO
1. Add in matplotlib to side panel -- COMPLETE
2. Make fake graph with random sliders and boxes and whatever to change the graph -- WIP
3. Integrate it into the main editor instance
4. Connect internal model values with the sliders and graph and make it "real"

In [1]:
%matplotlib inline
import jupyterlab_nodeeditor as jlne
from yggdrasil import yamlfile
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

In [2]:
filename = "X:\College\Grad School\Research\gc-Xyzic\jupyterlab_nodeeditor\examples\model_example.yml"
model_set = yamlfile.parse_yaml(filename, model_only=True)
model_sets = jlne.parse_yggdrasil_yaml(filename)

schema = yamlfile.get_schema()
socket_types = tuple(schema.form_schema['definitions']['schema']['definitions']['simpleTypes']['enum'])

ne = jlne.NodeEditor(socket_types = socket_types)

In [3]:
ne.socket_types = ('Temperature', 'Rainfall', 'Delta Time', 'Results')
ne.add_component(
    {"inputs": [
                {'title': 'Temperature Morning', 'key': 'temp1', 'socket_type': 'Temperature'},
                {'title': 'Temperature Afternoon', 'key': 'temp2', 'socket_type': 'Temperature'},
                {'title': 'Temperature Evening', 'key': 'temp3', 'socket_type': 'Temperature'}
                ],
     "outputs": [
                {'title': 'Results', 'key': 'results', 'socket_type': 'Temperature'}
                ],
     "title": "Temperature Averaging"
})

In [4]:
ne

In [7]:
m = widgets.FloatSlider(value = 0.0, min = -2.0, max = 2.0, step = 0.1, description = "Slope")
b = widgets.FloatSlider(value = 5.0, min = 0.0, max = 10.0, step = 0.1, description = "Intercept")
ne.node_editor.nodes[0].display_element.children += (m,)
ne.node_editor.nodes[0].display_element.children += (b,)

In [8]:
# Open initial sample plot png and add it to the sidebar
with open("test.png", "rb") as f:
    image = f.read()
    plotimg = widgets.Image(value = image, format = "png")

ne.node_editor.nodes[0].display_element.children += (plotimg,)

In [40]:
# Use widgets to let the sliders adjust the plot
def plot_change(change):
    # Create the graph
    fig, ax = plt.subplots()
    ax.set(xlabel='x', ylabel='y', title = "Simple Test Graph")
    ax.grid()
    x = np.linspace(-10, 10, num=1000)
    ax.set_ylim(0, 10)
    
    # TO-DO: Figure out how to work around this issue where Intercept syncs w slope but not vice versa
    # m.value = change["new"]
    
    # Plot the line
    ax.plot(x, m.value * x + b.value)
    
    # Save it as an image
    fig.savefig("slope.png")
    
    # display the image
    with open("slope.png", "rb") as f:
        new_plot = f.read()
        plotimg.value = new_plot
        
# Now have the sliders change the plot
m.observe(plot_change, "value")
b.observe(plot_change, "value")